In [1]:
# check working directory
!ls

bids	      fmriprep	       license.txt	 normalise_test.ipynb
config.json   fmriprepresult   nipype_notebooks  output
dcm2bids.log  joff2bids.ipynb  normalise.ipynb	 preprocessing_mz.ipynb


In [111]:
# Set the path to cbu2bids and First test run of dcm2bids on P1
!set path = $PATH:/imaging/local/software/cbu2bids
import glob, os
import pandas as pd
import numpy as np

fname = np.sort(glob.glob ('/mridata/cbu/*MR19002/*'))
# Specify subject list
subject_list=['01','02','04','05','03','06','07','08','09','10',\
             '11','12','13','14','15','16','18','17','26','20','19',
             '21','22','23','24','25','27','28','29']
cbu_dir = '/mridata/cbu/'
data_dir = [x.split('cbu/')[1] for x in fname[3:]] #The first 3 scans are pilot
data_dir = np.delete(data_dir,[2])
#data_dir=['CBU190317_MR19002/20190429_104638','CBU190337_MR19002/20190507_141324',\
#            'CBU190392_MR19002/20190523_143623','CBU190461_MR19002/20190614_153402']


29 CBU190699_MR19002/20190827_141931


In [201]:
no = 10
sub = subject_list[no]
print (sub, data_dir[no])

11 CBU190464_MR19002/20190617_105710


In [148]:
## Create Config file 
import json
def writeconfig(config, filename="config.json"):
    with open(filename, "w") as fp:
        json.dump(config, fp, indent=4)

In [202]:
# Write config object and write to file
config = dict(descriptions=[])
# so our first description might be
config["descriptions"] = []
config["descriptions"].append({"dataType": "anat", 
                               "suffix": "T1w",
                               "criteria": {
                                   "in": {
                                       "SeriesDescription": 
                                       "CBU_MPRAGE_32chn"
                                   }
                                   }
                               })


In [49]:
#Automatic coding functional run file into which AC or STM task
if 'vs' in glob.glob ('/imaging/mz01/fMRI_behavioural_data/p%s*MRI_1*'%sub)[0]:
    task_label=['vs','stm']
else:
    task_label=['stm','vs']
# add config info for the resting-state
config["descriptions"].append({"dataType": "func", 
                                  "suffix": "bold",
                                   "criteria": {
                                       "in": {
                                           "SeriesDescription": 
                                           "CBU_EPI_Standard_32chn_run1"
                                       }
                                   },
                                    "customHeader": {
                                      "TaskName": task_label[0]+'-run-01'
                                      }
                                  })
config["descriptions"].append({"dataType": "func", 
                                   "suffix": "bold",
                                   "criteria": {
                                       "in": {
                                           "SeriesDescription": 
                                           "CBU_EPI_Standard_32chn_run3"
                                       }
                                   },
                                    "customHeader": {
                                      "TaskName": task_label[0]+'-run-03'
                                      }
                                  })
config["descriptions"].append({"dataType": "func", 
                                   "suffix": "bold",
                                   "criteria": {
                                       "in": {
                                           "SeriesDescription": 
                                           "CBU_EPI_Standard_32chn_run5"
                                       }
                                   },
                                    "customHeader": {
                                      "TaskName": task_label[0]+'-run-05'
                                      }
                                  })
config["descriptions"].append({"dataType": "func", 
                                   "suffix": "bold",
                                   "criteria": {
                                       "in": {
                                           "SeriesDescription": 
                                           "CBU_EPI_Standard_32chn_run2"
                                       }
                                   },
                                    "customHeader": {
                                      "TaskName": task_label[1]+'-run-02'
                                      }
                                  }) 
config["descriptions"].append({"dataType": "func", 
                                   "suffix": "bold",
                                   "criteria": {
                                       "in": {
                                           "SeriesDescription": 
                                           "CBU_EPI_Standard_32chn_run4"
                                       }
                                   },
                                    "customHeader": {
                                      "TaskName": task_label[1]+'-run-04'
                                      }
                                  })
config["descriptions"].append({"dataType": "func", 
                                   "suffix": "bold",
                                   "criteria": {
                                       "in": {
                                           "SeriesDescription": 
                                           "CBU_EPI_Standard_32chn_run6"
                                       }
                                   },
                                    "customHeader": {
                                      "TaskName": task_label[1]+'-run-06'
                                      }
                                  })



In [203]:
#Appending info for FieldMaps (only sub-11 and onwards have these)
#Usually do this after func and anat images have been converted or intentedFor field wont work
echo1 = 0.00519
echo2 = 0.00765
func_files = glob.glob('output/sub-%s/func/*.nii.gz' %sub)
func_files = [x.split('output/sub-%s/' %sub)[1] for x in func_files]
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "magnitude1",
                               "criteria": {
                                   "in": {
                                       "SeriesDescription": "FieldMapping_32chn"
                                   },
                                    "equal": {
                                        "ImageType":
                                       ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
                                       "EchoTime": echo1
                                    }
                                   },
                                "customHeader": {
                                   "IntendedFor": func_files
                               
                               }
                              })
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "magnitude2",
                               "criteria": {
                                   "in": {
                                       "SeriesDescription": "FieldMapping_32chn"
                                   },
                                    "equal": {
                                        "ImageType":
                                       ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
                                       "EchoTime": echo2
                                   }
                               },
                                "customHeader": {
                                   "IntendedFor": func_files
                               }
                               
                              })
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "phasediff",
                               "criteria": {
                                   "in": {
                                       "SeriesDescription": "FieldMapping_32chn"
                                   },
                                   "equal": {
                                       "ImageType": ["ORIGINAL", "PRIMARY", "P", "ND"]
                                   }
                                   },
                               "customHeader": {
                                   "EchoTime1": echo1,
                                   "EchoTime2": echo2,
                                   "IntendedFor": func_files
                               }
                               })

In [121]:
# write and print config file again
writeconfig(config)
!cat config.json

{
    "descriptions": [
        {
            "dataType": "fmap",
            "suffix": "magnitude1",
            "criteria": {
                "in": {
                    "SeriesDescription": "FieldMapping_32chn"
                },
                "equal": {
                    "ImageType": [
                        "ORIGINAL",
                        "PRIMARY",
                        "M",
                        "ND",
                        "NORM"
                    ],
                    "EchoTime": 0.00519
                }
            },
            "customHeader": {
                "IntendedFor": [
                    "func/sub-28_task-stm-run-06_bold.nii.gz",
                    "func/sub-28_task-stm-run-04_bold.nii.gz",
                    "func/sub-28_task-vs-run-05_run-01_bold.nii.gz",
                    "func/sub-28_task-stm-run-02_bold.nii.gz",
                    "func/sub-28_task-vs-run-03_bold.nii.gz",
                    "func/sub-28_task-vs-run-05_run-02_bold.nii.g

In [204]:
# Third test run of dcm2bids on P1 with Functional info
!/imaging/local/software/cbu2bids/cbu2bids -d {cbu_dir+data_dir[no]} -p {sub} -c config.json -o output

Compression will be faster with 'pigz' installed
Chris Rorden's dcm2niiX version v1.0.20171215 GCC4.4.7 (64-bit Linux)
Found 3210 DICOM image(s)
Convert 128 DICOM as output/tmp_dcm2bids/sub-11/20190617_105710_AAHead_32_20190617105710_series013 (160x160x128x1)
slices not stacked: orientation varies (localizer?) [0.999544 0.0236297 -0.0187772 -0.0151015 0.93021 0.366718] != [0.999651 0.0163838 -0.0207314 -0.0125952 0.985151 0.171227]
Convert 1 DICOM as output/tmp_dcm2bids/sub-11/20190617_105710_AAHead_32_20190617105710_series041 (162x162x1x1)
slices not stacked: orientation varies (localizer?) [0.999651 0.0163838 -0.0207314 -0.0125952 0.985151 0.171227] != [0.999514 0.0240484 -0.0198407 -0.0256819 0.995907 -0.0866617]
Convert 1 DICOM as output/tmp_dcm2bids/sub-11/20190617_105710_AAHead_32_20190617105710_series041a (162x162x1x1)
Convert 1 DICOM as output/tmp_dcm2bids/sub-11/20190617_105710_AAHead_32_20190617105710_series041b (162x162x1x1)
Convert 32 DICOM as output/tmp_dcm2bids/sub-11/201

In [138]:
# now the output dir looks more interesting
!tree -C -I tmp_dcm2bids output

output
├── CHANGES
├── dataset_description.json
├── derivatives
├── participants.tsv
├── README
├── sub-01
│   ├── anat
│   │   ├── sub-01_T1w.json
│   │   └── sub-01_T1w.nii.gz
│   └── func
│       ├── sub-01_run_01_stm.tsv
│       ├── sub-01_run_02_vs.tsv
│       ├── sub-01_run_03_stm.tsv
│       ├── sub-01_run_04_vs.tsv
│       ├── sub-01_run_05_stm.tsv
│       ├── sub-01_run_06_vs.tsv
│       ├── sub-01_task-stm-run-01_bold.json
│       ├── sub-01_task-stm-run-01_bold.nii.gz
│       ├── sub-01_task-stm-run-03_bold.json
│       ├── sub-01_task-stm-run-03_bold.nii.gz
│       ├── sub-01_task-stm-run-05_bold.json
│       ├── sub-01_task-stm-run-05_bold.nii.gz
│       ├── sub-01_task-vs-run-02_bold.json
│       ├── sub-01_task-vs-run-02_bold.nii.gz
│       ├── sub-01_task-vs-run-04_bold.json
│       ├── sub-01_task-vs-run-04_bold.nii.gz
│       ├── sub-01_task-vs-run-06_bold.json
│       └── sub-01_task-vs-run-06_bold.nii.gz
├── sub-02
│   ├── anat
│   │   ├── sub-02_T1w.json
│   │   └─

In [123]:
# Check specification according to BIDS online validator
# http://bids-standard.github.io/bids-validator/


In [130]:
#Grab the list of FUNCTIONAL json files and load the data iteratively

fname = glob.glob('output/sub-%s/func/sub-%s*.json' %(sub,sub))

# Open the json file for the FUNCTIONAL image and load the data
import json
for f in fname:
    with open (f, 'r') as func:
        data = json.load(func)

    # Update values in the json dictionary
    data.update({'NumberOfVolumesDiscardedByUser': 0, 
                 'PulseSequenceType': 'Gradient Echo EPI',
                 'PulseSequenceDetails': 'Standard Siemens Sequence distributed with VD13D Software',
                 'NumberShots': 1,
                 'SliceEncodingDirection': 'j-',
                 'NonlinearGradientCorrection': False,
                 'InstitutionName': 'University of Cambridge', 
                 'InstitutionalDepartmentName':'Cognition and Brain Sciences Unit'})

    # Write the dictionary to the json file
    with open (f, 'w') as fun:
        json.dump(data, fun, indent=4)

### Save as subject general files to be located in the top directory

In [131]:
# Open the json file for the ANATOMICAL image and load the data
import json
with open ("output/sub-%s/anat/sub-%s_T1w.json" %(sub,sub), 'r') as anat:
    dat = json.load(anat)

# Update values in the json dictionary
dat.update({'PulseSequenceType': 'MPRAGE',
             'PulseSequenceDetails': 'Standard Siemens Sequence distributed with VD13D Software',
             'NonlinearGradientCorrection': False,
             'ParallelAcquisitionTechnique': 'GRAPPA',
             'PartialFourierDirection': 'j-',
             'PhaseEncodingDirection': 'j-',
             'InstitutionName': 'University of Cambridge', 
             'InstitutionalDepartmentName':'Cognition and Brain Sciences Unit'})

# Write the dictionary to the json file
with open ("output/sub-%s/anat/sub-%s_T1w.json" %(sub,sub), 'w') as ana:
    json.dump(dat, ana, indent=4)
    
### Save as subject general files to be located in the top directory

In [26]:
# Copy the subject folder to the BIDS directory
#!cp -avr output/sub-01 bids
#!cp -avr output/dataset_description.json bids
#!cp -avr output/participants.tsv bids
#!cp -avr output/README bids
#Doesn't work on my PC - permission id issue

In [125]:
#loop through each behavioral file to create task event file for each run
import scipy.io
beha_fname = glob.glob ('/imaging/mz01/fMRI_behavioural_data/*MRI*.csv')
pilot = glob.glob ('/imaging/mz01/fMRI_behavioural_data/*pilot*MRI*.csv')
beha_fname =  (set(beha_fname) - set(pilot)) #take out the pilot files from list
for f in beha_fname:
    #something about the column names is funky so renaming it
    if 'memory' in f:
        df = pd.read_csv(f, skiprows=1,header=None, names=['condition','response',\
            'answer','probe_ori','RT','no_click','x1','x2','ori1','ori2','ori3',\
            'ori4','ori5','ori6','ori7','ori8','ori9','load','retentionDur','ITI',\
            'fixation_onset','stimuli_onset','retention_onset','recall_onset','ITI_onset '])
        df['error'] = df.response-df.answer
        #readjust scale so error distribution is between [-180,180]
        df.loc[df.error>180, ['error']] = df.error-360
        df.loc[df.error<-180, ['error']] = df.error+360
        df.error = abs(df.error)
        df['correct'] = df.error<=90 #incorrect retrieval is defined as >90 degree error
        cond = 'stm'
    else:
        df = pd.read_csv(f, header=0)
        cond = 'vs'
    df['duration']= df['ITI_onset ']-df.fixation_onset
    basename = os.path.basename(f)
    #find subject number and run number
    sub_num = basename.split('p')[1][0:2]
    run_num = basename.split('MRI_')[1][0]
    outfile = '/imaging/mz01/fmriprep/output/sub-%s/func/sub-%s_run_0%s_%s.tsv'%(sub_num,sub_num,run_num,cond)
    try:
        df.to_csv(outfile,sep='\t',index=False,header=True)
    except:
        print("No sub-%s"%sub_num)
        pass


In [ ]:
### Save to /derivatives/fmriprep/

# Then run fmriprep on sub-01
# singularity run -C -B /imaging/jc01/kamitani:/kamitani 
# /imaging/local/software/singularity_images/fmriprep/fmriprep-1.1.8.simg 
# /kamitani/bids /kamitani/fmriprepnew participant --participant-label sub-03 -w
# /kamitani/fmriprepwork --nthreads 16 --omp-nthreads 16 --fs-license-file /kamitani/license.txt 
# --output-space T1w

# Run singularity with Bind (specify path /source:/dest) and Contain-all 
# Then reference the singularity fmriprep image
# Then the data input and output directory
# Then the participant option
# Then the participant label flag for processing a specific participant
# -w flag to specify a working directory
# --use-aroma for ICA denoising
# specify threads and memory, --mem_mb [MEM_MB]
# specify freesurfer license file, and disable
# specify output space T1w or {T1w, template} for MNI
## --write-graph for graph of workflow
## --sloppy for quick test run and --stop-on-first-crash
## --template {} to select 

!singularity run -C -B /imaging/mz01/fmriprep:/fmriprep /imaging/local/software/singularity_images/fmriprep/fmriprep-1.1.8.simg /fmriprep/output /fmriprep participant --participant-label 11 12 13 14 15 16 18 17 26 20 19 21 22 23 24 25 27 -v -w /fmriprep/fmriprepresult  --output-space template --resource-monitor --nthreads 16 --omp-nthreads 16 --fs-license-file /fmriprep/license.txt --fs-no-reconall 
    
# As Johan points out, you can then craft a command to run the code. 
# Once you've got that command, it's actually very straightforward to submit it to the clustejupyter r.
# For instance, using his example below, you could create a file called eg fmcluster.sh 
# which would contain two lines:
# #!/bin/sh
# /usr/bin/singularity run -C -B /imaging/jc01/kamitani :/kamitani 
# /imaging/local/software/singularity_images/fmriprep/fmriprep-1.1.8.simg 
# /kamitani/bids /kamitani/fmriprepnew participant --participant-label sub-03
# -w /kamitani/fmriprepwork --nthreads 16 --omp-nthreads 16 --fs-license-file
# /kamitani/license.txt --output-space T1w

# Then from the login node,
# sbatch fmcluster.sh
# would do the trick, although you might want to consider specifying the memory you need 
# (as noted  in the documentation)

## Run on login node took 2.5 hours with 8 processors


/usr/local/miniconda/lib/python3.6/site-packages/bids/grabbids/__init__.py:6: FutureWarning: grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8
  warnings.warn("grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8", FutureWarning)
/usr/local/miniconda/lib/python3.6/site-packages/bids/grabbids/__init__.py:6: FutureWarning: grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8
  warnings.warn("grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8", FutureWarning)
190905-14:33:25,95 nipype.workflow IMPORTANT:
	 
    Running fMRIPREP version 1.1.8:
      * BIDS dataset path: /fmriprep/output.
      * Participant list: ['11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27'].
      * Run identifier: 20190905-143325_caa50ee9-4954-495a-991d-140e6d5d6f5d.
    
190905-14:33:27,351 nipype.workflow IMPORTANT:
